# Qwen3-VL 모델 구축

AWS SageMaker Studio에서 Qwen3-VL 비전-언어 모델을 배포한다.

## 목차
1. 환경 설정 및 라이브러리 설치
2. AWS 리소스 확인
3. S3 버킷 설정
4. Qwen3-VL 모델 배포
5. 모델 테스트
6. 리소스 정리

## 주의사항
- GPU 인스턴스는 시간당 비용이 발생 (약 $1-2/시간)
- 사용 후 반드시 엔드포인트를 삭제
- 예상 소요 시간: 20-30분

---
## 1. 환경 설정 및 라이브러리 설치

필요한 라이브러리를 설치하고 버전을 확인합니다.

In [20]:
# 필수 라이브러리 설치
!pip install -q --upgrade pip
!pip install -q sagemaker>=2.200.0 boto3 transformers==4.45.0 accelerate torch torchvision
!pip install -q qwen-vl-utils pillow requests

In [18]:
# 설치 확인
import sagemaker
import boto3
import transformers
import sys

print("라이브러리 버전 확인:")
print(f"Python 버전: {sys.version}")
print(f"SageMaker SDK 버전: {sagemaker.__version__}")
print(f"Transformers 버전: {transformers.__version__}")
print(f"Boto3 버전: {boto3.__version__}")

라이브러리 버전 확인:
Python 버전: 3.12.9 | packaged by conda-forge | (main, Feb 14 2025, 08:00:06) [GCC 13.3.0]
SageMaker SDK 버전: 2.245.0
Transformers 버전: 4.37.0
Boto3 버전: 1.37.3


---
## 2. AWS 리소스 확인

SageMaker 세션, 실행 역할, 리전 정보를 확인합니다.

In [13]:
import sagemaker
import boto3
from datetime import datetime

# SageMaker 세션 생성
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

print("AWS 환경 정보:")
print(f"리전: {region}")
print(f"실행 역할 ARN: {role}")
print(f"세션 생성 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

AWS 환경 정보:
리전: ap-southeast-2
실행 역할 ARN: arn:aws:iam::565393029823:role/service-role/AmazonSageMaker-ExecutionRole-20251113T085559
세션 생성 시간: 2025-11-13 00:41:38


---
## 3. S3 버킷 설정

모델 아티팩트와 데이터를 저장할 S3 버킷을 설정합니다.

In [6]:
# 기본 S3 버킷 사용 (자동 생성)
bucket = sess.default_bucket()
prefix = 'qwen3-vl-demo'

print(f"S3 버킷: s3://{bucket}")
print(f"프리픽스: {prefix}")

S3 버킷: s3://sagemaker-ap-southeast-2-565393029823
프리픽스: qwen3-vl-demo


---
## 4. Qwen3-VL 모델 배포

### 4.1 모델 설정

HuggingFace Hub에서 Qwen3-VL 모델을 가져와 SageMaker 엔드포인트를 생성합니다.

In [ ]:
from sagemaker.huggingface import HuggingFaceModel
import json

# 모델 환경 변수 설정
hub_config = {
    'HF_MODEL_ID': 'Qwen/Qwen3-VL-2B-Instruct-FP8',  # Qwen3-VL FP8 양자화 모델
    'HF_TASK': 'image-to-text',
    'SM_NUM_GPUS': '1',
    # HuggingFace 토큰이 필요한 경우 아래 주석 해제
    # 'HF_TOKEN': 'your_huggingface_token_here',
}

# HuggingFace 모델 객체 생성
huggingface_model = HuggingFaceModel(
    env=hub_config,
    role=role,
    transformers_version="4.45",
    pytorch_version="2.1",
    py_version='py310',
    model_data=None  # HuggingFace Hub에서 직접 다운로드
)

print("모델 설정 완료!")
print(f"모델 ID: {hub_config['HF_MODEL_ID']}")
print(f"작업 유형: {hub_config['HF_TASK']}")

### 4.2 엔드포인트 배포

**중요**: 이 단계는 GPU 인스턴스를 시작하므로 비용이 발생합니다!
- 예상 배포 시간: 10-15분
- 예상 비용: ml.g5.xlarge 기준 약 $1.41/시간

In [ ]:
# 엔드포인트 이름 생성 (고유해야 함)
from datetime import datetime

endpoint_name = f"qwen-vl-endpoint-{datetime.now().strftime('%Y%m%d-%H%M%S')}"

print(f"엔드포인트 배포 시작...")
print(f"엔드포인트 이름: {endpoint_name}")
print(f"인스턴스 타입: ml.g5.xlarge")
print("\n배포 중...\n")

try:
    predictor = huggingface_model.deploy(
        initial_instance_count=1,
        instance_type="ml.g5.xlarge",  # GPU 인스턴스
        endpoint_name=endpoint_name,
    )
    
    print("\n엔드포인트 배포 완료!")
    print(f"엔드포인트 이름: {predictor.endpoint_name}")
    print(f"엔드포인트 URL: {predictor.endpoint}")
    
except Exception as e:
    print(f"\n배포 실패: {e}")
    print("\n문제 해결 팁:")
    print("1. 리전에 GPU 인스턴스 용량이 있는지 확인")
    print("2. IAM 역할에 필요한 권한이 있는지 확인")
    print("3. 다른 리전(예: us-east-1)에서 시도")

---
## 5. 모델 테스트

### 5.1 테스트 이미지 준비

In [8]:
from PIL import Image
import requests
from io import BytesIO
import base64

# 테스트 이미지 다운로드
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"

response = requests.get(image_url)
image = Image.open(BytesIO(response.content))

# 이미지 저장
image_path = "test_image.jpg"
image.save(image_path)

# 이미지 표시
print("테스트 이미지:")
display(image.resize((400, 300)))  # 크기 조정하여 표시

print(f"\n이미지 저장 완료: {image_path}")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:10                                                                                   │
│                                                                                                  │
│    7 image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-mad    │
│    8                                                                                             │
│    9 response = requests.get(image_url)                                                          │
│ ❱ 10 image = Image.open(BytesIO(response.content))                                               │
│   11                                                                                             │
│   12 # 이미지 저장                                                                               │
│   13 image_path = "test_image.jpg"                                                               │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/PIL/Image.py:3580 in open                                │
│                                                                                                  │
│   3577 │   for message in warning_messages:                                                      │
│   3578 │   │   warnings.warn(message)                                                            │
│   3579 │   msg = "cannot identify image file %r" % (filename if filename else fp)                │
│ ❱ 3580 │   raise UnidentifiedImageError(msg)                                                     │
│   3581                                                                                           │
│   3582                                                                                           │
│   3583 #                                                                                         │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7fe8da3ebfb0>

In [ ]:
# 이미지를 base64로 인코딩하는 함수
def encode_image_to_base64(image_path):
    """이미지 파일을 base64 문자열로 변환"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# 이미지 인코딩
image_base64 = encode_image_to_base64(image_path)
print("이미지 인코딩 완료")
print(f"Base64 길이: {len(image_base64)} 문자")

### 5.2 모델 추론 실행

In [ ]:
# 추론 요청 데이터 준비
payload = {
    "inputs": "이 이미지에 무엇이 보이나요? 자세히 설명해주세요.",
    "parameters": {
        "max_new_tokens": 512,
        "temperature": 0.7,
        "top_p": 0.9,
        "do_sample": True
    }
}

print("AI에게 질문:")
print(f"'{payload['inputs']}'\n")
print("추론 중...\n")

try:
    # 추론 실행
    response = predictor.predict(payload)
    
    print("추론 완료!\n")
    print("="*60)
    print("AI 응답:")
    print("="*60)
    print(response)
    print("="*60)
    
except Exception as e:
    print(f"추론 실패: {e}")
    print("\n문제 해결:")
    print("1. 엔드포인트가 'InService' 상태인지 확인")
    print("2. 이미지 형식이 올바른지 확인")
    print("3. CloudWatch 로그 확인")

### 5.3 추가 테스트 예제

In [ ]:
# 다양한 질문으로 테스트
test_questions = [
    "이 이미지의 색상 톤을 설명해주세요.",
    "이 사진이 어떤 계절에 촬영된 것 같나요?",
    "What objects can you see in this image?",  # 영어 질문도 가능
]

print("추가 테스트 진행 중...\n")

for i, question in enumerate(test_questions, 1):
    print(f"\n{'='*60}")
    print(f"질문 {i}: {question}")
    print(f"{'='*60}")
    
    try:
        response = predictor.predict({
            "inputs": question,
            "parameters": {"max_new_tokens": 256, "temperature": 0.7}
        })
        print(f"\n응답: {response}\n")
    except Exception as e:
        print(f"오류: {e}\n")

### 5.4 커스텀 이미지로 테스트

자신의 이미지를 업로드하여 테스트할 수 있습니다.

In [ ]:
# 파일 업로드 위젯 (Jupyter Lab/Notebook에서 실행)
from IPython.display import display
import ipywidgets as widgets

# 파일 업로드 위젯 생성
uploader = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description='이미지 선택'
)

print("아래 버튼을 클릭하여 이미지를 업로드하세요:")
display(uploader)

# 업로드된 이미지 처리 함수
def process_uploaded_image():
    if uploader.value:
        # 업로드된 파일 가져오기
        uploaded_file = list(uploader.value.values())[0]
        content = uploaded_file['content']
        
        # 이미지 저장
        with open('custom_image.jpg', 'wb') as f:
            f.write(content)
        
        # 이미지 표시
        image = Image.open('custom_image.jpg')
        display(image.resize((400, 300)))
        
        # 추론 실행
        question = input("\n이미지에 대해 질문하세요: ")
        response = predictor.predict({
            "inputs": question,
            "parameters": {"max_new_tokens": 512}
        })
        
        print(f"\n🤖 응답: {response}")
    else:
        print("⚠️ 먼저 이미지를 업로드하세요.")

# 수동으로 실행: process_uploaded_image()

---
## 6. 리소스 정리

**매우 중요!** GPU 인스턴스는 실행 중인 동안 계속 비용이 발생합니다.
작업이 끝나면 반드시 엔드포인트를 삭제하세요.

In [ ]:
# 현재 실행 중인 엔드포인트 확인
sm_client = boto3.client('sagemaker')

try:
    endpoint_info = sm_client.describe_endpoint(EndpointName=predictor.endpoint_name)
    print("📊 현재 엔드포인트 상태:")
    print(f"이름: {endpoint_info['EndpointName']}")
    print(f"상태: {endpoint_info['EndpointStatus']}")
    print(f"생성 시간: {endpoint_info['CreationTime']}")
    print(f"인스턴스 타입: {endpoint_info['ProductionVariants'][0]['InstanceType']}")
except Exception as e:
    print(f"엔드포인트 정보를 가져올 수 없습니다: {e}")

In [ ]:
# 엔드포인트 삭제
print("🗑️ 엔드포인트 삭제 중...")

try:
    predictor.delete_endpoint(delete_endpoint_config=True)
    print("✅ 엔드포인트 삭제 완료!")
    print("💰 비용 발생이 중단되었습니다.")
except Exception as e:
    print(f"❌ 삭제 실패: {e}")
    print("\n수동 삭제 방법:")
    print("1. AWS Console → SageMaker → 추론 → 엔드포인트")
    print(f"2. '{predictor.endpoint_name}' 선택")
    print("3. '삭제' 버튼 클릭")

In [ ]:
# 모델 삭제 (선택사항)
try:
    predictor.delete_model()
    print("✅ 모델 삭제 완료!")
except Exception as e:
    print(f"모델 삭제 중 오류 (무시 가능): {e}")

### 모든 엔드포인트 확인 및 정리

In [ ]:
# 현재 리전의 모든 엔드포인트 목록 확인
import pandas as pd

sm_client = boto3.client('sagemaker')
endpoints = sm_client.list_endpoints()['Endpoints']

if endpoints:
    print("📋 현재 실행 중인 엔드포인트 목록:\n")
    
    endpoint_list = []
    for ep in endpoints:
        endpoint_list.append({
            '이름': ep['EndpointName'],
            '상태': ep['EndpointStatus'],
            '생성시간': ep['CreationTime']
        })
    
    df = pd.DataFrame(endpoint_list)
    display(df)
    
    print("\n⚠️ 사용하지 않는 엔드포인트가 있다면 AWS Console에서 삭제하세요.")
else:
    print("✅ 실행 중인 엔드포인트가 없습니다.")

---
## 7. 비용 모니터링

AWS 비용을 확인하고 관리하는 방법입니다.

In [ ]:
# 예상 비용 계산기
from datetime import datetime, timedelta

# 인스턴스 타입별 시간당 가격 (미국 동부 기준, 2024년)
instance_prices = {
    'ml.g5.xlarge': 1.41,
    'ml.g5.2xlarge': 2.03,
    'ml.g5.4xlarge': 3.26,
    'ml.g5.12xlarge': 9.08,
}

def calculate_cost(instance_type, hours):
    """예상 비용 계산"""
    hourly_rate = instance_prices.get(instance_type, 0)
    total_cost = hourly_rate * hours
    return total_cost

# 사용 예시
instance_type = 'ml.g5.2xlarge'
usage_hours = 2  # 사용 시간

cost = calculate_cost(instance_type, usage_hours)
print(f"예상 비용:")
print(f"인스턴스 타입: {instance_type}")
print(f"사용 시간: {usage_hours}시간")
print(f"총 비용: ${cost:.2f}")
print(f"\n실제 비용은 리전과 사용 시간에 따라 다를 수 있습니다.")
print(f"정확한 비용은 AWS Billing Dashboard에서 확인하세요.")

---
## 8. 문제 해결 및 FAQ

### 자주 발생하는 오류와 해결 방법

**Q1: 엔드포인트 배포가 실패합니다.**

A: 다음을 확인하세요:
- 리전에 GPU 인스턴스 용량이 있는지 확인
- IAM 역할에 SageMaker 및 S3 권한이 있는지 확인
- 다른 리전(예: us-east-1, us-west-2)에서 시도

**Q2: 추론 결과가 이상합니다.**

A: 다음을 시도하세요:
- temperature 파라미터 조정 (0.1~1.0)
- max_new_tokens 증가
- 질문을 더 구체적으로 작성

**Q3: 비용이 너무 많이 나옵니다.**

A: 비용 절감 방법:
- 사용하지 않을 때는 즉시 엔드포인트 삭제
- 더 작은 인스턴스 타입 사용 (ml.g5.xlarge)
- AWS Budgets에서 예산 알림 설정

**Q4: HuggingFace 토큰이 필요한가요?**

A: Qwen2-VL 모델은 공개 모델이므로 대부분의 경우 토큰이 필요하지 않습니다.
만약 필요하다면:
1. https://huggingface.co 에서 계정 생성
2. Settings → Access Tokens에서 토큰 생성
3. hub_config에 'HF_TOKEN' 추가

---
## 9. 다음 단계

### 추가 학습 자료

- **AWS SageMaker 공식 문서**: https://docs.aws.amazon.com/sagemaker/
- **HuggingFace SageMaker 통합**: https://huggingface.co/docs/sagemaker/
- **Qwen 모델 정보**: https://huggingface.co/Qwen
- **AWS 비용 관리**: https://aws.amazon.com/ko/aws-cost-management/

### 개선 방향

- **배치 추론**: 여러 이미지를 한 번에 처리
- **Auto Scaling**: 트래픽에 따라 자동으로 인스턴스 증감
- **모델 최적화**: 양자화, 프루닝으로 추론 속도 개선
- **API Gateway 연동**: REST API로 서비스 제공

## ✅ 체크리스트
- [ ] 모든 라이브러리 설치 완료
- [ ] AWS 리소스 확인 완료
- [ ] S3 버킷 설정 완료
- [ ] Qwen3-VL 모델 배포 성공
- [ ] 모델 추론 테스트 완료
- [ ] **엔드포인트 삭제 완료** ⚠️ 필수!
- [ ] 실행 중인 리소스 없음 확인